# New Style HDMI input and Pixel Formatting

This notebook introduces the new features of PYNQ 2.0 for interacting with the video pipeline. The API has been completely
redesigned with high performance image processing applications in mind.

To start, download the base overlay and instantiate the HDMI input and output.

BLOCK 1 (Prepare HDMI - should see raw output on monitor after running this)

In [ ]:
from pynq.overlays.base import BaseOverlay
from pynq.lib.video import *

base = BaseOverlay("hls.bit")
hdmi_in = base.video.hdmi_in
hdmi_out = base.video.hdmi_out


# config hdmi
import copy
hdmi_in.configure()
hdmi_out_mode = copy.copy(hdmi_in.mode)
print(hdmi_in.mode)
print(hdmi_out_mode)

hdmi_out_mode.width = 1920
hdmi_out_mode.height = 1080
hdmi_out_mode.stride = 1920 * 3
hdmi_out_mode.fps = 60
hdmi_out.configure(hdmi_out_mode)

print(hdmi_out_mode.stride)

hdmi_in.start()
hdmi_out.start()

hdmi_in.tie(hdmi_out)


# start dma
from pynq import MMIO
import numpy as np
from pynq import allocate
print(hex(base.ip_dict['axi_dma_0']['phys_addr']))

# this line of code set the base address of dma control / status registers
#     base.ip_dict['axi_dma_0']['phys_addr'] : Query the physical address of the DMA axi_dma_0
#     0x60: The size of the DMA axi_dma_0's control / status registers
# please confirm the address of dma is set  correctly
axi_dma = MMIO(hex(base.ip_dict['axi_dma_0']['phys_addr']), 0x60)




# Read the status and control logic of dma
# print(hex(axi_dma.read(0)))     #MM2S control
# print(hex(axi_dma.read(4)))     #MM2S status

# print(hex(axi_dma.read(0x30)))  #S2MM control
# print(hex(axi_dma.read(0x34)))  #S2MM status

BLOCK2 (Start streaming)

In [5]:
_4k_byte = 3840 * 2160 * 3
_1k_byte = 1920 * 1080 * 3

# get the start address of the input 4K frame
f = hdmi_in.readframe()

# allocate and get the start address for the output 1080p frame
output_buffer = allocate(shape=(1080, 1920, 3),dtype=np.uint8)


# Read a 4K frame from DDR to PL, set signal from low offset to high offset to prevent masking
axi_dma.write(0, 1) # set Read channel to be active
axi_dma.write(0x1c, 0) # set Read channel to be active
axi_dma.write(0x18, f.device_address) # set the start address of the input frame
axi_dma.write(0x28, _4k_byte) # set the length of the input frame

# Write a 1080p frame from PL to DDR, set signal from low offset to high offset to prevent masking
axi_dma.write(0x30, 1) # set Write channel to be active
axi_dma.write(0x4c, 0) # set Write channel to be active
axi_dma.write(0x48, output_buffer.device_address) # set the start address of the output frame
axi_dma.write(0x58, _1k_byte) # set the length of the output frame


# # wait for Read channel to finish (check the registers with 0x04 shift)
# # Since their will be leftover data in the PL pipeline, it will finish earlier than the Write channel
# while hex(axi_dma.read(4)) != "0x1002":
#               pass


import time
num_frames = 1200
start = time.time()
for _ in range(num_frames):

    # get the start address of the next input 4K frame
    f = hdmi_in.readframe()

    # wait for Read channel to finish (check the registers with 0x04 shift)
    # Since their will be leftover data in the PL pipeline, it will finish earlier than the Write channel
    while hex(axi_dma.read(4)) != "0x1002":
              pass
    
    # Since current Read channel has finished, we can start the next Reading for the next frame, it can help flush the PL pipeline
    # and make sure the current write channel will finish. (Receive the leftover data in the pipeline)
    axi_dma.write(0, 1)
    axi_dma.write(0x1c, 0)
    axi_dma.write(0x18, f.device_address)
    axi_dma.write(0x28, _4k_byte)


    # wait for Write channel to finish (check the registers with 0x34 shift)
    while hex(axi_dma.read(0x34)) != "0x1002":
              pass
        
    
    # Since current Write channel has finished, the output frame is ready, we can send it to the HDMI output
    # this line is triggering the VDMA to send the frame to HDMI output, HDMI output will accept at 60 fps, and will
    # stall VDMA if VDMA produce higher data rate.
    # This line is of blocking and will wait for the HDMI output to finish and it can control the frame rate (60 fps)
    hdmi_out.writeframe(output_buffer)
    

    # Since current output frame has been sent to HDMI output, it is not valid anymore, we can start writing the next frame
    # to its address. 
    axi_dma.write(0x30, 1)
    axi_dma.write(0x4c, 0)
    axi_dma.write(0x48, output_buffer.device_address)
    axi_dma.write(0x58, _1k_byte)


    
    
end = time.time()
print("Frames per second:  " + str(num_frames / (end - start)))

BLOCK3 (Save board output)

In [ ]:
import PIL.Image
import numpy as np
img = PIL.Image.fromarray(output_buffer)
img.save("board.png")
np.save("board.npy", output_buffer)
img

BLOCK4 (Close HDMI)

In [13]:
hdmi_out.close()
hdmi_in.close()